In [ ]:
!pip install imblearn

     |████████████████████████████████| 206 kB 26.8 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import *
from sklearn.metrics         import *
from imblearn.pipeline import Pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.preprocessing   import OneHotEncoder
from sklearn.preprocessing   import LabelEncoder
from   sklearn.compose            import *
from   sklearn.ensemble           import RandomForestClassifier, ExtraTreesClassifier, IsolationForest
from   sklearn.experimental       import enable_iterative_imputer
from   sklearn.impute             import *
from   sklearn.linear_model       import LogisticRegression, PassiveAggressiveClassifier, RidgeClassifier, SGDClassifier
from   sklearn.metrics            import roc_auc_score # We have not covered it yet in class. The basics - AUC is from 0 to 1 and higher is better.
#from   sklearn.pipeline           import Pipeline
from   sklearn.preprocessing      import *
from   sklearn.tree               import *
from   sklearn.metrics            import balanced_accuracy_score
from   sklearn.feature_selection import *

In [ ]:
from sklearn.ensemble import * 

In [ ]:
rgg_rss_data = pd.read_excel('RGG_RSS_final.xlsx')

In [ ]:
rgg_rss_data

,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,4 yrd,5 yrd,6 yrd,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key
0,Recycle,S-HEIL,C,0,0,1,2,2,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,5128_geary,NaN,False
1,Recycle,S-HEIL,C,0,0,0,0,1,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,5620_geary,NaN,False
2,Recycle,S-HEIL,I,0,0,0,0,1,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,1947_clement,NaN,False
3,Recycle,S-HEIL,C,0,0,1,0,1,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,1919_clement,NaN,False
4,Recycle,S-HEIL,C,0,0,1,0,0,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,1909_clement,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,Compost,R-HEIL,I,0,0,0,1,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,1220_jon,NaN,False
12822,Compost,R-HEIL,I,0,0,0,3,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,1221_jon,NaN,False
12823,Compost,R-HEIL,C,0,0,0,2,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,1000_california,NaN,False
12824,Compost,R-HEIL,IC,0,0,0,2,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,250_eavenworth,NaN,False


In [ ]:
rgg_rss_data['Commodity'].unique()

array(['Recycle', 'Garbage/Compost', 'Garbage', 'Compost',
       'Garbage/Recycle'], dtype=object)

## Removing Outliers

In [ ]:
X=rgg_rss_data[['Commodity', '16 gal', '20 gal', '32 gal', '64 gal', '96 gal','1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', '#Units',
       'Company', 'Has Key']]
y=rgg_rss_data['Time']

X=X.replace('na',np.nan)
X=X.replace('nan',np.nan)
X=X.replace('.',np.nan)

preprocessing = ColumnTransformer([
                            ('scaler',StandardScaler(),[1,2,3,4,5,6,7,8,9,10,11,12,13]),                       
                            ('categorical', OneHotEncoder(handle_unknown='ignore'), [0,14,15])
                            ])

pipe = Pipeline([
                #('col_selection',FunctionTransformer(categorical_col_transform,validate=False)),
                ('imputation',SimpleImputer(missing_values=np.nan,strategy="most_frequent")),
                ('preprocessing',preprocessing),
                ('Isol',IsolationForest(contamination=0.1))]) 


yhat = pipe.fit_predict(X)

mask = yhat != -1
X, y = X[mask], y[mask]

In [ ]:
data=rgg_rss_data[mask]

In [ ]:
dt = DecisionTreeRegressor(max_depth=8,
             min_samples_leaf=0.13,
            random_state=3)

In [ ]:
col=list(data.columns)
time_index=col.index("Time")
#print(col.index("Time"))

In [ ]:
data=rgg_rss_data.loc[rgg_rss_data['Outlier']==0]

In [ ]:
X=data
y=data['Time']

In [ ]:
mode_day=X['Day'].mode()[0]
X['Day'].replace('na',mode_day,inplace=True)
X['Day'].replace('nan',mode_day,inplace=True)
X['Day']=X['Day'].fillna(mode_day)


/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/series.py:4512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
X['Day'].unique()

array([2, 5, 3, 1, 4, 'na', nan, 7], dtype=object)

In [ ]:
X=X.drop(columns=['Inside/Curb','Vehicle Type','Meandor','Route','Tipper','Outlier','Address_Street','Code','Time'])
X

,Commodity,16 gal,20 gal,32 gal,64 gal,96 gal,Day,#Units,1 yrd,1.5 yrd,2 yrd,3 yrd,4 yrd,5 yrd,6 yrd,Company,Truck,Neighborhood,Has Key
0,Recycle,0,0,1,2,2,2.0,5,0,0,0,0,0,0,0,RSS,SL,Inner Richmond,False
1,Recycle,0,0,0,0,1,2.0,1,0,0,0,0,0,0,0,RSS,SL,Inner Richmond,False
2,Recycle,0,0,0,0,1,2.0,1,0,0,0,0,0,0,0,RSS,SL,Inner Richmond,False
3,Recycle,0,0,1,0,1,2.0,2,0,0,0,0,0,0,0,RSS,SL,Inner Richmond,False
4,Recycle,0,0,1,0,0,2.0,1,0,0,0,0,0,0,0,RSS,SL,Inner Richmond,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,Compost,0,0,0,1,0,4.0,1,0,0,0,0,0,0,0,RGG,RL,Chinatown,False
12822,Compost,0,0,0,3,0,4.0,3,0,0,0,0,0,0,0,RGG,RL,Chinatown,False
12823,Compost,0,0,0,2,0,4.0,2,0,0,0,0,0,0,0,RGG,RL,Chinatown,False
12824,Compost,0,0,0,2,0,4.0,2,0,0,0,0,0,0,0,RGG,RL,Chinatown,False


In [ ]:
X_dummy=pd.get_dummies(X,columns=['Commodity','Company','Company','Truck','Neighborhood','Has Key','Day'])

In [ ]:
X_dummy

,16 gal,20 gal,32 gal,64 gal,96 gal,#Units,1 yrd,1.5 yrd,2 yrd,3 yrd,...,Neighborhood_West Twin Peaks,Neighborhood_Western Addition,Has Key_False,Has Key_True,Day_1.0,Day_2.0,Day_3.0,Day_4.0,Day_5.0,Day_7.0
0,0,0,1,2,2,5,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,0,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,0,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0,0,1,0,1,2,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
12822,0,0,0,3,0,3,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
12823,0,0,0,2,0,2,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
12824,0,0,0,2,0,2,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [ ]:
y

0        118.0
1         59.0
2         86.0
3         41.0
4         31.0
         ...  
12821     98.0
12822    100.0
12823     66.0
12824    118.0
12825    180.0
Name: Time, Length: 11543, dtype: float64

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_dummy, y, test_size = 0.2, random_state=16)

In [ ]:
dt = DecisionTreeRegressor(max_depth=8,
             min_samples_leaf=0.13,
            random_state=3)

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
y_pred = dt.predict(X_validation)

In [ ]:
mse_dt = MSE(y_validation, y_pred)


In [ ]:
rmse_dt = mse_dt**(1/2)

In [ ]:
print("Test set RMSE of dt: {:.2f}".format(rmse_dt))

Test set RMSE of dt: 56.15


### Random Forest Regressor

In [ ]:
data

,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,4 yrd,5 yrd,6 yrd,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key
0,Recycle,S-HEIL,C,0,0,1,2,2,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,5128_geary,NaN,False
1,Recycle,S-HEIL,C,0,0,0,0,1,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,5620_geary,NaN,False
2,Recycle,S-HEIL,I,0,0,0,0,1,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,1947_clement,NaN,False
3,Recycle,S-HEIL,C,0,0,1,0,1,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,1919_clement,NaN,False
4,Recycle,S-HEIL,C,0,0,1,0,0,NaN,912,...,0,0,0,RSS,0,SL,Inner Richmond,1909_clement,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,Compost,R-HEIL,I,0,0,0,1,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,1220_jon,NaN,False
12822,Compost,R-HEIL,I,0,0,0,3,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,1221_jon,NaN,False
12823,Compost,R-HEIL,C,0,0,0,2,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,1000_california,NaN,False
12824,Compost,R-HEIL,IC,0,0,0,2,0,NaN,302,...,0,0,0,RGG,0,RL,Chinatown,250_eavenworth,NaN,False


In [ ]:
X=data[['Commodity', '16 gal', '20 gal', '32 gal', '64 gal', '96 gal','1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', '#Units', 'Neighborhood',
       'Company', 'Has Key']]
y=data['Time']

In [ ]:
X=X.replace('na',np.nan)
X=X.replace('nan',np.nan)
X=X.replace('.',np.nan)

In [ ]:
categorical_cols = (X.dtypes == object)
preprocessing = ColumnTransformer([('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
                                   ('continuous', StandardScaler(), -categorical_cols)])  

pipe = Pipeline([
                #('col_selection',FunctionTransformer(categorical_col_transform,validate=False)),
                ('imputation',SimpleImputer(missing_values=np.nan,strategy="most_frequent")),
                ('preprocessing',preprocessing),
                ('RF',RandomForestRegressor())]) 


In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state=16)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# Minimum number of samples required to split a node
min_samples_split = [10,15,20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10,15,20]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
random_grid = {'RF__n_estimators': n_estimators,
               'RF__max_features': max_features,
               'RF__max_depth': max_depth,
               'RF__min_samples_split': min_samples_split,
               'RF__min_samples_leaf': min_samples_leaf,
               'RF__bootstrap': bootstrap}

In [ ]:
rf_random = RandomizedSearchCV(estimator = pipe, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('imputation',
                                              SimpleImputer(strategy='most_frequent')),
                                             ('preprocessing',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               Commodity        True
16 gal          False
20 gal          False
32 gal          False
64 gal          False
96 gal          False
1 yrd           False
1.5 yrd         False
2 yrd           False
3 yrd           False
4 yrd           False
5 yrd           False
6 yrd           False
#Units          False
Nei...
                                                                    min_samples_leaf=5,
                                                                    m

In [ ]:
pipe=rf_random.best_estimator_

In [ ]:
predictions = pipe.predict(X_validation)

In [ ]:
print("accuracy:",1-mean_absolute_percentage_error(predictions,y_validation))

accuracy: 0.6434566595092768


In [ ]:
for x,y in zip(predictions,y_validation):
    print(x,y)

83.9703492257627 72.0
67.61176404032402 60.0
260.28485887665084 252.0
55.0313947107922 49.0
98.82220165082855 96.0
83.44135274569693 60.0
55.468615590174984 101.0
47.188702697062936 31.0
57.10526242447091 45.0
109.30224392873129 62.0
26.54448550888623 21.0
69.79627582320978 82.0
98.65909611988408 70.0
122.39460947540789 92.0
83.92758842405154 65.0
88.76973204175036 101.0
110.66389927860006 80.0
68.11289389296275 67.0
44.82803390617749 51.0
55.922554042857875 42.0
46.73024979859614 90.0
105.16378545900808 73.0
113.24020621067315 129.0
110.2752912544592 117.0
75.11049154289779 76.0
29.854397683278027 19.0
105.16378545900808 101.0
35.723104092409706 26.0
201.85428509351075 294.0
128.9144876497891 99.0
62.1773031435626 58.0
38.6964789790686 31.0
57.10526242447091 82.0
134.33002587786393 221.0
96.50815659810236 89.0
40.43890208782979 46.0
171.83366594400366 162.0
83.9703492257627 108.0
104.61296719652898 274.0
119.37480892421914 68.0
39.92330857544555 44.0
137.8657577581959 120.0
64.7581731

## XGBoost

In [ ]:
categorical_cols = (X.dtypes == object)
preprocessing = ColumnTransformer([('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
                                   ('continuous', StandardScaler(), -categorical_cols)])  

pipe = Pipeline([
                #('col_selection',FunctionTransformer(categorical_col_transform,validate=False)),
                ('imputation',SimpleImputer(missing_values=np.nan,strategy="most_frequent")),
                ('preprocessing',preprocessing),
                ('XGB',GradientBoostingRegressor())]) 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]
# Minimum number of samples required to split a node
min_samples_split = [10,15,20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10,15,20]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Method of selecting loss on regression
loss=['ls','huber', 'quantile']
# Stopping the tree early
n_iter_no_change=[None,3]
## Learning the rate
learning_rate=np.linspace(0.1,5,num=5)

In [ ]:
random_grid = {'XGB__n_estimators': n_estimators,
               'XGB__max_features': max_features,
               'XGB__max_depth': max_depth,
               'XGB__min_samples_split': min_samples_split,
               'XGB__min_samples_leaf': min_samples_leaf,
               'XGB__loss':loss,
               'XGB__n_iter_no_change':n_iter_no_change,
               'XGB__learning_rate':learning_rate}

In [ ]:
xg_random = RandomizedSearchCV(estimator = pipe, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
xg_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/model_selection/_search.py:926: RuntimeWarning: overflow encountered in square
  array_means[:, np.newaxis]) ** 2,


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('imputation',
                                              SimpleImputer(strategy='most_frequent')),
                                             ('preprocessing',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               Commodity        True
16 gal          False
20 gal          False
32 gal          False
64 gal          False
96 gal          False
1 yrd           False
1.5 yrd         False
2 yrd           False
3 yrd           False
4 yrd           False
5 yrd           False
6 yrd           False
#Units          False
Nei...
                   param_distributions={'XGB__learning_rate': array([0.1  , 1.325, 2.55 , 3.775, 5.   ]),
                                        'XGB__loss'

In [ ]:
pipe=xg_random.best_estimator_

In [ ]:
predictions = pipe.predict(X_validation)

In [ ]:
print("accuracy:",1-mean_absolute_percentage_error(predictions,y_validation))

accuracy: 0.6438335364152368


In [ ]:
for x,y in zip(predictions,y_validation):
    print(x,y)

## Rough Code (only for reference)

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
#rfr = RandomForestRegressor(n_estimators=200)
#rfr.fit(X_train, y_train)
#print(rfr.score(X_train, y_train))
#print(rfr.score(X_validation, y_validation))

In [ ]:
rfr.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
hyperparameter={'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 60,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 97,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
rfr_tuned = RandomForestRegressor(**hyperparameter)
rfr.fit(X_train, y_train)
print(rfr.score(X_train, y_train))
print(rfr.score(X_validation, y_validation))

0.7622743814296966
0.5609163644589128


In [ ]:
rfr = RandomForestRegressor(n_estimators=200)
rfr.fit(X_train, y_train)
print(rfr.score(X_train, y_train))
print(rfr.score(X_validation, y_validation))

In [ ]:
a=[]
for x,y in zip(X_train.columns,rfr.feature_importances_):
    a.append((x,y))

In [ ]:
a.sort(key=lambda x:x[1],reverse=True)
a

[('96 gal', 0.4096645808023728),
 ('#Units', 0.19991315715912256),
 ('64 gal', 0.042709197684657),
 ('Neighborhood_Nob Hill', 0.02941212375495386),
 ('32 gal', 0.022212005056344207),
 ('Day_1', 0.018146247539570812),
 ('Neighborhood_Financial District', 0.01698957676219288),
 ('Neighborhood_Pacific Heights', 0.015540764958431687),
 ('Has Key_True', 0.01541004285533351),
 ('Commodity_Recycle', 0.015057466434372167),
 ('Has Key_False', 0.014687521512077924),
 ('Commodity_Garbage', 0.013876096322805344),
 ('Truck_RL', 0.013765121083179156),
 ('Neighborhood_Russian Hill', 0.013617875712022965),
 ('Day_3', 0.013596913704812162),
 ('Neighborhood_Inner Richmond', 0.013210766709418804),
 ('Day_4', 0.012891238400451947),
 ('Commodity_Garbage/Compost', 0.011177282176837172),
 ('Truck_SL', 0.010788760477722125),
 ('Commodity_Garbage/Recycle', 0.010761486451962362),
 ('Day_2', 0.01069726027851928),
 ('Day_5', 0.010206879363004896),
 ('Neighborhood_North Beach', 0.009248293863825821),
 ('16 gal', 0

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 60, stop = 400, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}



In [ ]:
rf = RandomForestRegressor()
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [60, 97, 135, 173, 211,
                                                         248, 286, 324, 362,
                                                         400]},
                   random_state=42, verbose=2)

In [ ]:
rf_random.best_estimator_.get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 60,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 97,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
evaluate()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=938c6ad9-491d-4307-bf8a-c751a244ce4f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>